# Training experiments

In this code, we will train different models to explore different options for solving the problem.

## Import libraries and data

In [226]:
import mlflow
from datetime import datetime
import awswrangler as wr
import random
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from mlflow import MlflowClient

Export the necessary environment variables for working with Minio.

In [227]:
# Export environment variables
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [228]:
mlflow_server = "http://localhost:5000"

mlflow.set_tracking_uri(mlflow_server)

Load the processed data from Minio.


In [229]:
X_train = wr.s3.read_csv("s3://data/train/bike_sharing_demand_X_train_scaled.csv").values
y_train = wr.s3.read_csv("s3://data/train/bike_sharing_demand_y_train.csv").values.ravel()
X_test = wr.s3.read_csv("s3://data/test/bike_sharing_demand_X_test_scaled.csv").values
y_test = wr.s3.read_csv("s3://data/test/bike_sharing_demand_y_test.csv").values.ravel()

# Model training

In [230]:
def get_or_create_experiment(experiment_name):
    """
    Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

    Parameters:
    - experiment_name (str): Name of the MLflow experiment.

    Returns:
    - str: ID of the existing or newly created MLflow experiment.
    """

    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)

Initialize the mlflow experiment

In [231]:
experiment_id = get_or_create_experiment("Bike Sharing Demand")

print(f"Experiment ID: {experiment_id}")

Experiment ID: 1


In [232]:
run_name_parent = "best_hyperparam_"  + datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

Show the size of the training and test sets

In [233]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((12165, 20), (12165,), (5214, 20), (5214,))

Data normalization is an important step before training the model. 

In [234]:
# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 150], 
    #'max_depth': [None, 10, 15],
    #'bootstrap': [True, False]
}

# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor()

# Set up Grid Search with 5-fold cross-validation
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')

# Start the MLflow run
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Perform the grid search and fit the model
    grid_search_rf.fit(X_train, y_train)
    
    # Get the best model from the grid search
    best_rf_model = grid_search_rf.best_estimator_
    
    # Make predictions using the best model
    rf_predictions = best_rf_model.predict(X_test)
    
    # Calculate metrics
    mse_rf = mean_squared_error(y_test, rf_predictions)
    rmse_rf = mean_squared_error(y_test, rf_predictions, squared=False)
    mae_rf = mean_absolute_error(y_test, rf_predictions)
    r2_rf = r2_score(y_test, rf_predictions)
    
    # Log the best parameters and metrics to MLflow
    mlflow.log_param("best_rf_n_estimators", best_rf_model.n_estimators)
    mlflow.log_param("best_rf_max_depth", best_rf_model.max_depth)
    mlflow.log_param("best_rf_min_samples_split", best_rf_model.min_samples_split)
    mlflow.log_param("best_rf_min_samples_leaf", best_rf_model.min_samples_leaf)
    mlflow.log_param("best_rf_bootstrap", best_rf_model.bootstrap)
    
    mlflow.log_metric("rf_mse", mse_rf)
    mlflow.log_metric("rf_rmse", rmse_rf)
    mlflow.log_metric("rf_mae", mae_rf)
    mlflow.log_metric("rf_r2", r2_rf)
    
    # Get the first row of the test set and log it to MLflow
    input_example = X_test[0:1]
    
    # Define the artifact path
    artifact_path = "best_rf_model"
    
    # Infer the schema of the input example
    signature = mlflow.models.infer_signature(X_train, best_rf_model.predict(X_train))
    
    # Log the best Random Forest model to the MLflow server
    mlflow.sklearn.log_model(
        sk_model=best_rf_model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name='bike_sharing_model_dev',
        metadata={'model_data_version': 1}
    )
    
    # Get the URI of the logged model
    model_uri = mlflow.get_artifact_uri(artifact_path)
    
    # Print the results
    print(f"Best Random Forest model logged with MSE: {mse_rf}, RMSE: {rmse_rf}, MAE: {mae_rf}, R²: {r2_rf}")
    print(f"Best Random Forest parameters: {grid_search_rf.best_params_}")

/home/chris/bike-sharing-demand/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/chris/bike-sharing-demand/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:16: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/chris/bike-sharing-demand/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils 

Best Random Forest model logged with MSE: 0.14847120183602538, RMSE: 0.3853196099811498, MAE: 0.25733299972348506, R²: 0.9329953227239205
Best Random Forest parameters: {'n_estimators': 100}


## Testing the model

In [235]:
loaded_model = mlflow.sklearn.load_model(model_uri)

In [236]:
X_test = np.array(X_test)

In [237]:
# Get a random element from test set
input_example = X_test[random.randint(0, X_test.shape[0])] 

print(f"Input example: {input_example}")

Input example: [-1.003541   -0.16956604  0.68381845  0.7351559   1.3580773  -0.81836506
  1.67734303 -0.29878575 -0.01282315 -0.58212799  1.68342244 -0.56997785
 -0.40854189 -0.39931795 -0.40771959  2.44527012 -0.40840489 -0.4120971
 -1.27528153  0.58022529]


In [238]:
int(np.exp(loaded_model.predict(input_example.reshape(1, -1))))

/tmp/ipykernel_101376/3514142192.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int(np.exp(loaded_model.predict(input_example.reshape(1, -1))))


209

## Register the model

In [241]:
client = MlflowClient()

name = "bike_sharing_model_prod"
desc = "Production model for bike sharing demand prediction"

client.create_registered_model(name=name, description=desc)

tags = best_rf_model.get_params()
tags["model"] = type(best_rf_model).__name__
tags["mse"] = mse_rf

result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

client.set_registered_model_alias(name, "champion", result.version)

2024/08/25 03:54:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: bike_sharing_model_prod, version 1
